In [ ]:
"""
Pinecone에 RAG 데이터 업로드 
- 입력: data/rag_marketing_data.csv
- 출력: Pinecone index ("bigcon")에 임베딩 벡터 삽입
"""

import os
import pandas as pd
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

# .env 파일에서 API 키 불러오기
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY","API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENVIRONMENT", "us-east-1")  # 환경값은 콘솔에서 확인
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME", "bigcon")

if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY 환경 변수가 설정되지 않았습니다.")

In [21]:
# 1. CSV 로드
df = pd.read_csv(r"C:\Users\rladl\Desktop\bigcontest_2025\2025_shcard_bigcontest\data\rag_marketing_data.csv")

print(f"✅ CSV 로드 완료: {len(df)} rows")

✅ CSV 로드 완료: 7 rows


In [34]:
# 2. Pinecone 초기화

# Pinecone 클라이언트 초기화 
pc = Pinecone(api_key=PINECONE_API_KEY)
print("✅ Pinecone 클라이언트 초기화 완료")

# 기존 인덱스 확인
existing_indexes = pc.list_indexes().names()
print(f"📋 기존 인덱스: {existing_indexes}")

# # 기존 인덱스 없을 시 - 인덱스 생성 또는 확인
# if INDEX_NAME not in pc.list_indexes().names():
#     pc.create_index(
#         name=INDEX_NAME,
#         dimension=1024,
#         metric='cosine',
#         spec=ServerlessSpec(
#             cloud='aws',
#             region='us-east-1'
#         )
#     )
#     print(f"📦 새 Pinecone 인덱스 생성: {INDEX_NAME}")
# else:
#     print(f"✅ '{INDEX_NAME}' 인덱스가 이미 존재합니다")

# 새 인덱스 만들기
INDEX_NAME="my-index"

pc.create_index(
    name=INDEX_NAME,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


# 6. 인덱스 연결 및 상태 확인
index = pc.Index(INDEX_NAME)
stats = index.describe_index_stats()

print(f"\n📊 '{INDEX_NAME}' 인덱스 정보:")
print(f"   - 총 벡터 수: {stats.get('total_vector_count', 0):,}")
print(f"   - 차원 수: {stats.get('dimension', 'N/A')}")
print(f"   - 인덱스 사용률: {stats.get('index_fullness', 0)*100:.2f}%")

✅ Pinecone 클라이언트 초기화 완료
📋 기존 인덱스: ['boazadv', 'boazadvproject', 'bigcon']

📊 'my-index' 인덱스 정보:
   - 총 벡터 수: 0
   - 차원 수: 768
   - 인덱스 사용률: 0.00%


In [35]:
# 4. 임베딩 모델 초기화


# [사용X] Google - 유료
# import google.generativeai as genai

# # 환경 변수 설정
# GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"] # 환경 변수에서 불러오기

# # Google AI 설정
# genai.configure(api_key=GOOGLE_API_KEY)

# # 임베딩 모델 초기화
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=GOOGLE_API_KEY  # 직접 전달
# )

# print("✅ GoogleGenerativeAIEmbeddings 초기화 완료")


# [사용O] HuggingFace - 무료

from langchain_community.embeddings import HuggingFaceEmbeddings

# 예시: all-MiniLM-L6-v2 (무료, 384차원)
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
print("✅ HuggingFaceEmbeddings 초기화 완료")

✅ HuggingFaceEmbeddings 초기화 완료


In [36]:
# 5. 저장할 데이터 로드 - 텍스트와 메타데이터 준비
# 현재 코드에는 따로 텍스트 분할 과정 X => 필요시 text_splitter 사용

texts = df["content"].tolist()
metadatas = df.to_dict(orient="records")

print(f"📄 준비된 문서 수: {len(texts)}개")
print(f"📋 메타데이터 컬럼: {df.columns.tolist()}")

📄 준비된 문서 수: 7개
📋 메타데이터 컬럼: ['id', 'category', 'title', 'content', 'source']


In [40]:
# 6. Pinecone VectorStore 생성 및 데이터 업로드

print("🚀 Pinecone 인덱싱 시작...")
vectorstore = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    index_name=INDEX_NAME
)

print(f"✅ 업로드 완료! 총 {len(texts)}개 문서가 Pinecone 인덱스에 저장되었습니다.")


# 9. 업로드 결과 확인
index = pc.Index(INDEX_NAME)
stats = index.describe_index_stats()

print(f"\n📊 업로드 후 인덱스 상태:")
print(f"   - 총 벡터 수: {stats.get('total_vector_count', 0):,}")
print(f"   - 차원 수: {stats.get('dimension', 'N/A')}")
print(f"   - 인덱스 사용률: {stats.get('index_fullness', 0)*100:.2f}%")

🚀 Pinecone 인덱싱 시작...
✅ 업로드 완료! 총 7개 문서가 Pinecone 인덱스에 저장되었습니다.

📊 업로드 후 인덱스 상태:
   - 총 벡터 수: 0
   - 차원 수: 768
   - 인덱스 사용률: 0.00%


In [44]:
# 10. 업로드 테스트 (간단한 검색)
if vectorstore:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    test_docs = retriever.get_relevant_documents("치킨집 홍보 방법")
    
    print(f"\n🔍 테스트 검색 결과: {len(test_docs)}개 문서")
    for i, doc in enumerate(test_docs[:2]):  # 처음 2개만 출력
        print(f"   {i+1}. {doc.page_content[:100]}...")


🔍 테스트 검색 결과: 3개 문서
   1. 대학생 상권 분식집은 인스타그램 챌린지를 통해 고객이 음식 사진을 업로드하면 무료 음료를 제공했다. 참여 게시물 1,000건 이상 기록하며 신규 고객 유입에 성공했다....
   2. 대학생 상권 분식집은 인스타그램 챌린지를 통해 고객이 음식 사진을 업로드하면 무료 음료를 제공했다. 참여 게시물 1,000건 이상 기록하며 신규 고객 유입에 성공했다....


- 6-1. 대용량 데이터 저장 시 사용

In [ ]:
# 6-1. 배치 처리를 위한 함수 (대용량 데이터 처리용)
# def upload_texts_in_batches(texts, metadatas, embeddings, index_name, batch_size=100):
#     """대용량 데이터를 배치 단위로 업로드"""
    
#     vectorstore = None
#     total_batches = (len(texts) + batch_size - 1) // batch_size
    
#     for i in range(0, len(texts), batch_size):
#         batch_texts = texts[i:i+batch_size]
#         batch_metadatas = metadatas[i:i+batch_size]
        
#         current_batch = (i // batch_size) + 1
#         print(f"🔄 배치 {current_batch}/{total_batches} 처리 중... ({len(batch_texts)}개 문서)")
        
#         if vectorstore is None:
#             # 첫 번째 배치로 vectorstore 생성
#             vectorstore = PineconeVectorStore.from_texts(
#                 texts=batch_texts,
#                 embedding=embeddings,
#                 metadatas=batch_metadatas,
#                 index_name=index_name
#             )
#         else:
#             # 기존 vectorstore에 추가
#             vectorstore.add_texts(
#                 texts=batch_texts,
#                 metadatas=batch_metadatas
#             )
        
#         print(f"✅ 배치 {current_batch} 완료")
    
#     return vectorstore

# # 8. 데이터 크기에 따른 업로드 방식 선택
# if len(texts) <= 100:
#     # 소량 데이터: 한 번에 업로드
#     print("🚀 Pinecone 인덱싱 시작 (소량 데이터)...")
    
#     vectorstore = PineconeVectorStore.from_texts(
#         texts=texts,
#         embedding=embeddings,
#         metadatas=metadatas,
#         index_name=INDEX_NAME
#     )
    
#     print(f"✅ 업로드 완료! 총 {len(texts)}개 문서가 Pinecone 인덱스에 저장되었습니다.")
    
# else:
#     # 대량 데이터: 배치 처리
#     print("🚀 Pinecone 인덱싱 시작 (대량 데이터 - 배치 처리)...")
    
#     vectorstore = upload_texts_in_batches(
#         texts=texts,
#         metadatas=metadatas,
#         embeddings=embeddings,
#         index_name=INDEX_NAME,
#         batch_size=100
#     )
    
#     print(f"✅ 배치 업로드 완료! 총 {len(texts)}개 문서가 Pinecone 인덱스에 저장되었습니다.")


# # 9. 업로드 결과 확인
# index = pc.Index(INDEX_NAME)
# stats = index.describe_index_stats()

# print(f"\n📊 업로드 후 인덱스 상태:")
# print(f"   - 총 벡터 수: {stats.get('total_vector_count', 0):,}")
# print(f"   - 차원 수: {stats.get('dimension', 'N/A')}")
# print(f"   - 인덱스 사용률: {stats.get('index_fullness', 0)*100:.2f}%")

# # 10. 업로드 테스트 (간단한 검색)
# if vectorstore:
#     retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
#     test_docs = retriever.get_relevant_documents("테스트 검색")
    
#     print(f"\n🔍 테스트 검색 결과: {len(test_docs)}개 문서")
#     for i, doc in enumerate(test_docs[:2]):  # 처음 2개만 출력
#         print(f"   {i+1}. {doc.page_content[:100]}...")
